In [1]:
from loadmodules import *
from pylab import *
from const import *
import numpy as np
import string as str
import bisect
from scipy.interpolate import interp1d, interp2d
from scipy import stats
from parallel_decorators import vectorize_parallel
import gc

In [2]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx,item in enumerate(seq) if item in seen or seen_add(item)]

def get_accreted_insitu_particles( runs, dirs, outpath, treepaths, mergertree=False, dobfs=True, verbose=False, writehdf5=True ):
       

        for run in range( len( runs ) ):
                print("<<Doing run %s >>"%runs[run])
                path = dirs[run] + runs[run]
                print('in path ',dirs[run])
                #fexp = path + '/expfac_snapshotlist_%s.txt' % runs[run]
                #fexp = '/u/rgrand/expfaclists/' + '/expfac_snapshotlist_%s.txt' % runs[run]
                #print('fpath,runs=',fpath,runs)
                #fexp = fpath[run] + '/expfac_snapshotlist_%s.txt' % runs[run]
                #expansion_fact = np.loadtxt(fexp)

                # get expansion factor list from snapshot header
                expansion_fact = []
                nsnapshots = 0
                while gadget_snapexists( nsnapshots, snappath=path+'/output/' ):
                    tempsnap = gadget_readsnap( nsnapshots, snappath=path+'/output/', onlyHeader=True)
                    expansion_fact.append(tempsnap.time)
                    nsnapshots += 1

                expansion_fact = np.array(expansion_fact)
                print("expansion_fact=",expansion_fact,len(expansion_fact))

                snap0 = nsnapshots - 1

                #wpath = outpath[run] + runs[run] + '/idlists'
                wpath = outpath[run] + '/lists' + '/accretedstardata/' + runs[run] 
                if not os.path.exists(wpath):
                    print('Making directory ',wpath)
                    os.makedirs(wpath)  

                ntotstar = 0
                ntotacc = 0
                ntotins = 0
                ntotfofs =0

                
                if mergertree:
                    treebase = 'trees_sf1_%03d' % snap0
                    # treepath = '%s/mergertrees/Au-%s/%s' % (dirs[run],runs[run].split('_')[-1],treebase)
                    treepath = '%s/%s' % (treepaths[run],treebase)
                    # treepath = '/%s/%s/%s'%(treepaths[run],runs[run],treebase)
                    #treepath = '%s/Au-%s/%s'%(treepaths[run],runs[run].split('_')[-1],treebase)
                    #treepath = '/virgotng/mpa/Auriga/level4/Variants//mergertrees/Au-18_3000snaps/%s'%treebase 

                    #treepath = '/u/rgrand/ggcode/mergertree_code/level4_1e11msun/%s/%s'%(runs[run],treebase)
                    print('treepath=',treepath)
                    t = load_tree( 0, 0, base=treepath, verbose=False )                    

                    
                    snap_numbers_main,redshifts_main,subfind_indices_main,first_progs_indices_main,ff_tree_indices_main,fof_indices_main,prog_mass_main,next_prog_indices = t.return_first_next_mass_progenitors(0)
                    print('subfind_indices_main,first_progs_indices_main,ff_tree_indices_main,fof_indices_main,prog_mass_main=',subfind_indices_main,first_progs_indices_main,ff_tree_indices_main,fof_indices_main,prog_mass_main)
                    print('snap_numbers_main=',snap_numbers_main)
                else:
                    fof_indices_main = np.zeros(snap0)
                    subfind_indices_main = np.zeros(snap0)
                    
                loadptype = [4]
                """
                if verbose: # verify that subhalo number in tree is cumulative
                    for num in range(0,5):
                        indy, = np.where( (t.data['sgnr'] == num) & (t.data['snum']==51) )
                        print('fof=',num)
                        isort = np.argsort(t.data['sbnr'][indy])
                        print('subhalo indicies =',t.data['sbnr'][indy][isort])
                """
                s = gadget_readsnap( snap0, snappath=path+'/output/', hdf5=True, loadonlytype=loadptype, loadonly=['pos', 'vel', 'mass', 'age', 'id'])
                sf = load_subfind( snap0, dir=path+'/output/', hdf5=True, loadonly=['fpos', 'frc2', 'frm2', 'svel', 'flty', 'fnsh', 'slty', 'smty'])#, cosmological=True )

                # understand subfind indices
                s.center = sf.data['fpos'][0,:]
                print('fpos=',sf.data['fpos'][0,:]) 
                s.centerat( s.center )

                if 1 in loadptype:
                        idmb = s.get_most_bound_dm_particles()

                radius = sf.data['frc2'][0] # R200 crit
                radius = sf.data['frm2'][0] # R mean
                print('Calculating accreted particle information for all star particles within %f kpc of the main halo centre.'%(radius*1e3))

                # Selection star IDs within virial radius
                bigage = np.zeros(s.nparticlesall.sum())
                st = s.nparticlesall[:4].sum()
                en = np.int_(st + s.nparticlesall[4])
                bigage[st:en] = s.data['age'].astype('float64')
                print('star birth scale factors=',bigage)

                istars, = np.where( (s.r() < radius) & (s.type == 4) & (bigage>0.) )

                starid = s.data['id'][istars]#.astype('uint64')
                
                print('starid=',starid)
                #pos = s.pos[istars,:].astype('float64') 
                #sradius = np.sqrt( ( s.pos[istars,:]**2 ).sum(axis=1) )
                #id_inside = s.data['id'][istars]
                allradius = s.r()

                mainsub_ids = s.data['id'][s.nparticlesall[:4].sum():np.int_(s.nparticlesall[:4].sum()+sf.data['slty'][0,4])]

                ## Get star ids bound in subhalo
                print('Calculating stars in satellites at final snapshot...')
                last_star = 0
                sub_ids = []
                nfofs = len(sf.data['fnsh'])
                nfofs = 1000
                fpos = sf.data['fpos'] - sf.data['fpos'][0]
                ifofs, = np.where( (np.sqrt(np.sum(fpos**2, axis=1)) < 2.*radius ) )
                print('fpos=',fpos)
                print("ifofs=",ifofs,len(ifofs))
                nfofs = ifofs[-1]
                print('doing nfofs=',nfofs)
                for ll in range(nfofs):   
                    offset = s.nparticlesall[:4].sum() + sf.data['flty'][:ll,4].sum()
                    nsub_thisfof = sf.data['fnsh'][ll]   
                    #first_star = np.int_(offset)
                    first_star = offset
                    if sf.data['flty'][:ll,4].sum() > first_star:
                        raise IndexError("index overlap at beginning of fof %d"%ll)

                    print('Looking in fof %d (out of %d) with %s subhaloes (snap %d)'%(ll,len(sf.data['fnsh']),nsub_thisfof,snap0))
                    for l in range(nsub_thisfof):         
                        print('doing sub & fof:',l,ll)    
                        subind = l + sf.data['fnsh'][:ll].sum()           
                        nstarinsubhalo = sf.data['slty'][subind,4]  
                        if first_star < last_star:
                            raise IndexError("first_star < last_star in fof %d, sub %d"%(ll,l))
                        #last_star = first_star + np.int_(nstarinsubhalo)
                        last_star = first_star + nstarinsubhalo
                        
                        subhalodmmass = sf.data['smty'][subind,1]
                        subhalostarmass = sf.data['smty'][subind,4]
                        subhalototmass = subhalodmmass + subhalostarmass
                        dmfrac = subhalodmmass / subhalototmass

                        idtmp = s.data['id'][np.int_(first_star):np.int_(last_star)]#.astype('int64')
                        

                        if ll == 0 and subind == 0:
                            first_star = last_star
                            continue
                        elif dmfrac > 0.1:
                            inside, = np.where( (allradius[np.int_(first_star):np.int_(last_star)] < radius) & (bigage[np.int_(first_star):np.int_(last_star)] > 0.) )
                            if len(inside):
                                sub_ids.extend(idtmp[inside])
                        first_star = last_star
                        
                        
                sub_ids = np.array(sub_ids)
                print('len sub_ids=',len(sub_ids))
                print('... done.')
                ##
                
                nstars = size(istars)
                pn = np.arange(nstars)
                mydict = dict(zip(starid,pn))

                alist = bigage[istars]
                #print('alist=',alist,alist.min())
                age = s.cosmology_get_lookback_time_from_a( alist, is_flat=True )
                #print('age=',age)
                out = np.digitize(alist, bins=expansion_fact)
                print('alist=',alist,min(alist))
                
                #indub, = np.where(np.isin(starid, idballs)==True)
                #print('ages&snaps=',age[indub],out[indub])

                minsnap = min(out)
                
                maxsnap = snap0

                print('minsnap,maxsnap=',minsnap,maxsnap)
                
                out[out>maxsnap] = maxsnap

                srad = np.zeros(nstars)

                star = 0
                foundtot = 0

                print('Total star number', nstars)
                
                snaps = np.arange(minsnap, maxsnap+1)
                
                # Get snap list from unique list of out snaps
                snaps = np.unique(out)

                # Cycle through snaps to first find when each starid is first found in the main halo
                found_mask = np.array(np.ones(len(starid)))
                #boundmain_firstsnap = np.array(np.zeros(len(starid)))
                #id_fb = np.array(np.zeros(len(starid)))

                starid_tmp = starid
                print('Cycling through %d star IDs'%np.size(starid_tmp))
                print('starid_tmp=',starid_tmp)
                #print('starid=',starid)
                
                if dobfs:
                    print('Entering calculation for first main bound snapshot.')
                    boundmain_firstsnap = []
                    id_fb = []
                    for isnap, snap in enumerate(snaps):
                        print('FIND FIRST BOUND SNAP: Doing snap ',snap)
                        psf = load_subfind( snap, dir=path+'/output/', hdf5=True, loadonly=['fpos', 'frc2', 'svel', 'flty', 'fnsh', 'slty'] )
                        if not 'slty' in psf.data:
                            print('WARNING: slty field not in subfind object data list. Passing to the next snapshot.')
                            continue
                    
                        p = gadget_readsnap( snap, snappath=path+'/output/', hdf5=True, loadonlytype=[4], loadonly=['id'], subfind=psf)#, uintIDs=uintIDs)#, loadonlyhalo=0)

                        try:
                            fof0 = fof_indices_main[maxsnap-snap]
                            sub0 = subfind_indices_main[maxsnap-snap]
                        except:
                            print('Cannot find fof and subfind index from tree at this snapshot.')
                            fof0 = fof_indices_main[0]
                            sub0 = subfind_indices_main[0]

                        print("*fof0,sub0 selected from merger tree=",fof0,sub0)
                    
                        first_star = np.int_( p.nparticlesall[:4].sum() + psf.data['flty'][:fof0,4].sum() + psf.data['slty'][psf.data['fnsh'][:fof0].sum():sub0,4].sum() )
                        last_star = np.int_( first_star + psf.data['slty'][sub0,4] )
                        print('first_star,last_star=',first_star,last_star,snap)

                        main_ids = p.data['id'][ first_star:last_star ]

                        print('starid_tmp, main_ids=',starid_tmp, main_ids)
                        boundmain_index = np.isin(starid_tmp, main_ids)
                        #print('boundmain_index=',boundmain_index,np.size(np.where(boundmain_index==True)))
                        #print('snap,ids=',snap,starid_tmp[boundmain_index])
                        boundmain_firstsnap.extend([snap]*np.size(np.where(boundmain_index==True)))
                        id_fb.extend(starid_tmp[boundmain_index])

                        starid_tmp = starid_tmp[boundmain_index==False]

                        print('Size of starid_tmp=',np.size(starid_tmp))
                 
                        del psf
                        del p
                    

                    boundmain_firstsnap.extend([-1]*np.size(starid_tmp))
                    id_fb.extend(starid_tmp)
                
                    boundmain_firstsnap = np.array(boundmain_firstsnap)
                    id_fb = np.array(id_fb)

                    isort = np.argsort(id_fb)
                    boundmain_firstsnap = boundmain_firstsnap[isort]
                    id_fb = id_fb[isort]
                
                    #total_found = np.size(np.where(found_mask==0))
                    print('boundmain_firstsnap=',boundmain_firstsnap)
                    #print('We have assigned %d (out of a total of %d) particles a snapshot corresponding to the first time they are bound to the main halo'%(total_found,len(starid)))
                else:
                    # Only for testing
                    boundmain_firstsnap = np.ones(len(starid))
                    id_fb = np.ones(len(starid))
                
                
                pnf = 0
                fuzztmp = []
                fuzzsub = []
                fuzzsubsnap = []
                fuzzsubid = []
                idmissingsum = 0

                k = 0
                
                
                print("Starting parallel loop over snaps=",snaps,len(snaps))
                if mergertree:
                    get_indices( snaps, maxsnap, wpath, path, runs[run], out, starid, fof_indices_main, subfind_indices_main, t, mainsub_ids, sub_ids, boundmain_firstsnap, id_fb, expansion_fact, dobfs=dobfs, verbose=verbose, writehdf5=writehdf5)
                else:
                    get_indices( snaps, maxsnap, wpath, path, runs[run], out, starid, fof_indices_main, subfind_indices_main, None, mainsub_ids, sub_ids, boundmain_firstsnap, id_fb, expansion_fact, dobfs=dobfs, verbose=verbose, writehdf5=writehdf5 )

In [ ]:
@vectorize_parallel(method='processes', num_procs=10)
def get_indices( snap, maxsnap, wpath, path, run, out, starid, fof_indices_main, subfind_indices_main, t, mainsub_ids, sub_ids, boundmain_firstsnap, id_fb, expansion_fact, dobfs=True, verbose=True, writehdf5=True):

    s = gadget_readsnap( snap, snappath=path+'/output/', hdf5=True, loadonlytype=[4], loadonly=['pos', 'mass', 'id'], forcesingleprec=False)
    sf = load_subfind( snap, dir=path+'/output/', hdf5=True, loadonly=['fnsh', 'spos', 'shmr', 'slty', 'smty', 'flty'] )

    # check that we have subfind data for this snapshot first, and if not, skip it
    try:
        nsub = sf.data['fnsh']
    except:
        print('No subfind data for snapshot %d. Skipping.'%snap)
        return

    if verbose:
            print('Doing snapshot %4d of %4d' % (snap, maxsnap))
            print("Time=",s.time)

    Time = s.cosmology_get_lookback_time_from_a( s.time, is_flat=True )
            
    iii, = np.where( out == snap )
    sid = starid[iii]
    print('Looking for IDs:',sid)
    idlist_true = []

    idlist_accfofs = []
    sublistfofs = []
    idlist_acc = []
    sublist = []
    idapplist_tmp = []
    fflag = []
    offsetsub = 0

    idins = []
    idacc = []
    snapins = []
    snapacc = []
    subacc = []
    fofflag = []
    idfuzz = []

    try:
        fof0 = fof_indices_main[maxsnap-snap]
        sub0 = subfind_indices_main[maxsnap-snap]
        
    except:
        fof0 = fof_indices_main[0]
        sub0 = subfind_indices_main[0]
        print('Failed here: maxsnap, snap=',maxsnap,snap,len(fof_indices_main),len(subfind_indices_main))
        
    
    print("*fof0,sub0 selected from merger tree=",fof0,sub0)
    
    nsub_cum = 0
    last_star = 0
    for ll in range(len(sf.data['fnsh'])):   
        offset = s.nparticlesall[:4].sum() + sf.data['flty'][:ll,4].sum()
        nsub_thisfof = sf.data['fnsh'][ll]   
        #first_star = np.int_(offset)
        first_star = offset
        if sf.data['flty'][:ll,4].sum() > first_star:
            raise IndexError("index overlap at beginning of fof %d"%ll)

        #print('Looking in fof %d (out of %d) with %s subhaloes (snap %d)'%(ll,len(sf.data['fnsh']),nsub_thisfof,snap))
        for l in range(nsub_thisfof):             
            subind = l + sf.data['fnsh'][:ll].sum()           
            nstarinsubhalo = sf.data['slty'][subind,4]  
            if first_star < last_star:
                raise IndexError("first_star < last_star in fof %d, sub %d"%(ll,l))
            #last_star = first_star + np.int_(nstarinsubhalo)
            last_star = first_star + nstarinsubhalo
            
            subhalodmmass = sf.data['smty'][subind,1]
            subhalostarmass = sf.data['smty'][subind,4]
            subhalototmass = subhalodmmass + subhalostarmass
            dmfrac = subhalodmmass / subhalototmass

            idtmp = s.data['id'][np.int_(first_star):np.int_(last_star)]#.astype('int64')
            if verbose:
                if len(idtmp):
                    print('idtmp=',idtmp)
                    print('sid=',sid)

            
            if ll == fof0 and subind == sub0:
                if verbose:
                    print('*first_star,last_star,snap=',first_star,last_star,snap)
                index_ins = np.where(np.isin( sid, idtmp)==True)
                #print('intersect=',np.intersect1d(sid, idtmp))
                #indub, = np.where(np.isin(idballs,idtmp)==True)
                #if np.size(indub) > 0:
                #    print('indub=',idballs[indub])
                #    print('******** %d found in snap %d'%(np.size(indub),snap))
                #    print('fof0,sub0=',fof0,sub0)
                if np.size(index_ins)>0:
                    idlist_true.extend(sid[index_ins])
            else:
                index_acc = np.where(np.isin( sid, idtmp)==True)
                #indub, = np.where(np.isin(idballs, idtmp)==True)
                #if np.size(indub) > 0:
                #    print('indub=',idballs[indub])
                #    print('>>>>>>> %d found in snap %d'%(np.size(indub),snap))
                #    print('fof0,sub0=',ll,subind)
                if verbose:
                    print('index_acc=',index_acc)
                if np.size(index_acc)>0:
                    if dmfrac > 0.1:
                        idlist_acc.extend(list(sid[index_acc]))
                        sublist.extend([subind]*len(sid[index_acc]))
                        fflag.extend([ll]*len(sid[index_acc]))
                    else:
                        idlist_true.extend(list(sid[index_acc]))

            first_star = last_star

        last_star = first_star
        nsub_cum += nsub_thisfof
        idtmp = s.data['id'][np.int_(last_star):np.int_(sf.data['flty'][:ll+1,4].sum())]

        index_acc = np.where(np.isin( sid, idtmp)==True)
        
        if np.size(index_acc)>0:
            #print "**fuzz stars found=",np.size(index_acc)
            idlist_acc.extend(list(sid[index_acc]))
            sublist.extend([0]*len(sid[index_acc])) # should this be 0?
            fflag.extend([-1]*len(sid[index_acc]))

    # look in end of snapshot fuzz
    idtmp = s.data['id'][sf.data['flty'][:,4].sum():s.nparticlesall[4]]

    index_acc = np.where(np.isin( sid, idtmp)==True)
    if np.size(index_acc)>0:
        idlist_acc.extend(list(sid[index_acc]))
        sublist.extend([0]*len(sid[index_acc]))
        fflag.extend([-1]*len(sid[index_acc]))

    # deal with not-found yet particles here
    idfound = np.concatenate((idlist_true,idlist_acc))
    
    print("number found (%d/%d) this snap="%(np.size(idfound),np.size(sid)),snap)
    print("this snap accreted=",np.size(idlist_acc),snap)
    print("this snap in-situ=",np.size(idlist_true),snap)

    idins.extend(idlist_true[:])
    snapins.extend([snap]*len(idlist_true))
    idacc.extend(idlist_acc[:])
    snapacc.extend([snap]*len(idlist_acc))       
    subacc.extend(sublist[:])
    fofflag.extend(fflag[:])


    idins = np.array(idins)
    snapins = np.array(snapins)
    idacc = np.array(idacc)
    snapacc = np.array(snapacc)
    subacc = np.array(subacc)
    fofflag = np.array(fofflag)

    del s
    del sf

    s = gadget_readsnap( snap, snappath=path+'/output/', onlyHeader=True)
    
    """
    # This is happens when the main fof and sub number are not indexed as zero in subfind
    wrong_index, = np.where( (fofflag == 0) & (subacc == 0) )
    print('number of accreted particles with fof and sub = 0=',np.size(wrong_index))
    snaps_wrong = np.unique(snapacc[wrong_index])
    print('snaps_wrong=',snaps_wrong)
    """
    
    ### write to file ###
    #n_ins = size(idins)
    #n_acc = size(idacc)

    if t:
    
        nextprogid = []
        peakmassid = []
        """Fisrt, find the root of the branch (earliest Tree index) for each star particle, beginning with the Tree index 
        coresponding to the snapshot, group and subhalo number in which they are born"""
        rootid = -np.ones(len(snapacc)).astype('int')
        for j, sub in enumerate(np.unique(subacc)):
            fofs = np.unique( fofflag[subacc==sub] ).astype('i4')
            for fof in fofs:
                if fof == -1:
                    continue
                tree_index, = np.where( (t.data['snum'] == snap) & (t.data['sbnr'] == sub) & (t.data['sgnr'] == fof) )
                if not tree_index:
                    print('tree_index=',tree_index)
                    print('WARNING: Could not find this accreted progenitor in the Tree. Snap %d, sub %d, fof %f, run %s'%(snap,sub,fof,run))

                else:
                    indy, = np.where( (subacc == sub) & (fofflag == fof) )
                    rootid[ indy ] = tree_index[0]

                    if verbose:
                        print('sub,snap=',sub,snap)
                        print('tree_index,sub,snap,fof=',tree_index,t.data['sbnr'][tree_index[0]],t.data['snum'][tree_index[0]],t.data['sgnr'][tree_index[0]])
                if verbose:
                    print('rootid=',rootid[indy],subacc[indy])

        ii, = np.where( rootid == -1 )
        if verbose:
            print('are they any -1s:',np.size(np.where(rootid==-1)))
            print('the rest=',np.size(np.where(rootid!=-1)))
            print('FOF of rootid -1: ',np.unique(fofflag[ii]),len(ii))
        
        nextprogid = []
        peakmassid = []

        snap_numbers_main,redshifts_main,subfind_indices_main,first_progs_indices_main,ff_tree_indices_main,fof_indices_main,prog_mass_main,next_prog_indices = t.return_first_next_mass_progenitors( 0 )

        prog_mass_main = np.array(prog_mass_main)
        if verbose:
            print('snap_numbers,fof0,sub0=',snap_numbers_main,fof_indices_main,subfind_indices_main)
            print('prog_mass_main,next_prog_indices=',prog_mass_main,next_prog_indices)
        
        for i, fval in enumerate(snap_numbers_main):
            nextprog = next_prog_indices[i].astype('int')
            while nextprog != -1:
                #try:
                snap_numbers,redshifts,subfind_indices,first_progs_indices,ff_tree_indices,fof_indices,prog_mass,dum = t.return_first_next_mass_progenitors( nextprog )
                prog_mass = np.array(prog_mass)
                if verbose:
                    print('nextprog=',nextprog)
                    print('prog_mass=',prog_mass)
                    print('first_progs_indices bef=',first_progs_indices)
                desc = t.data['desc'][first_progs_indices[0]]
                while desc != -1 and desc not in first_progs_indices_main:
                    prog_mass = np.append(prog_mass, t.data['smty'][desc][4])
                    first_progs_indices = np.append(first_progs_indices, desc)
                    desc = t.data['desc'][desc]

                # debug
                if np.isnan(np.sum(prog_mass)):
                    print('Nan error')

  
                indy = np.argsort(prog_mass)
                if verbose:
                    print('first_progs_indices=',first_progs_indices)
                    print('prog_mass=',prog_mass,nextprog)
                    print('indy=',indy,nextprog)
                    print('first_progs_indices[indy[-1]]=',first_progs_indices[indy[-1]],nextprog)
                peakmassid.append( first_progs_indices[indy[-1]] )
                                    
                nextprogid.append( nextprog )
                #except:
                #    print('Mywarning: We have not been able to set up all next prog ids.')

                nextprog = t.data['npin'][nextprog]

        nextprogid = np.array(nextprogid)
        peakmassid = np.array(peakmassid)
        if verbose:
            print('nextprogid=',nextprogid,len(nextprogid))

        pkmassid = -np.ones(len(rootid)).astype('int')
        pkmassid_infalltime = -np.ones(len(rootid)).astype('f8')
        
        # Check that the rootid of none of these accreted particles is in the main progenitor branch. This should not (and does not appear) to happen, but will set the flag the ids of such particles as -9 if true.
        wrong_rootidindex = np.isin(rootid, first_progs_indices_main)
        #print('wrong_rootidindex=',wrong_rootidindex,np.size(wrong_rootidindex))
        #print('wrong rootids,snap=',rootid[wrong_rootidindex],subacc[wrong_rootidindex],snapacc[wrong_rootidindex],fofflag[wrong_rootidindex],snap)
        #print('plist bef=',np.array(list(sorted(set(list(rootid))))))
        plist = np.array(list(sorted(set(list(rootid[ ~wrong_rootidindex ])))))
        if len(wrong_rootidindex):
            pkmassid[ wrong_rootidindex ] = -9
            pkmassid_infalltime[ wrong_rootidindex ] = -9
            rootid[ wrong_rootidindex ] = -9

        #print('pkmassid bef=',pkmassid)
        
        #plist = np.array(list(sorted(set(list(rootid)))))
        ###
        
        #print('plist=',plist,len(plist))
                            
        for did in plist:
            #print('doing did=',did)
            cnt = 0
            infcnt = 0
            infalltime = 0
            #disttomain = []
            #desc = did
            # need to go up the tree first (if we can)
            snap_numbers,redshifts,subfind_indices,first_progs_indices,ff_tree_indices,fof_indices,prog_mass,dum = t.return_first_next_mass_progenitors( did )
            try:
                desc = first_progs_indices[-1]
            except:
                desc = did
            # then we go down
            while desc != -1:
                if t.data['desc'][desc] == first_progs_indices_main[ snap_numbers_main==t.data['snum'][t.data['desc'][desc]] ] and t.data['desc'][desc] != -1:
                    break
                desc = t.data['desc'][desc]
                dx = t.data['spos'][desc] - t.data['spos'][first_progs_indices_main[ snap_numbers_main==t.data['snum'][desc]] ]
                disttomain = np.sqrt(np.nansum( dx**2 ))
                if disttomain < t.data['frc2'][first_progs_indices_main[ snap_numbers_main==t.data['snum'][desc] ]] and infcnt == 0:
                    infalltime = 1./ (1. + t.data['reds'][desc])
                    infcnt = 1
                #print('infalltime=',infalltime)
                    
                cnt += 1
                #if cnt > maxsnap+1:
                #    break
            if verbose:
                print('desc=',desc)
            if desc != -1:
                pkmassid[ rootid==did ] = peakmassid[ nextprogid==desc ]

            else:
                if verbose:
                    print('The rootid %d has been assigned a pkmassid of -1 for now'%did)

            index_tinf, = np.where(rootid==did)
            #print('index_tinf=',index_tinf)
            pkmassid_infalltime[ rootid==did ] = infalltime #np.array([infalltime]*len(np.where(rootid==did)))
            #print('pk infalltime=',infalltime,pkmassid_infalltime[ rootid==did ])


        if np.size(np.where(pkmassid==-1)) > 0:
            iind, = np.where( pkmassid == -1 )
            if verbose:
                print('iind=',iind)
                print('fofflag=',fofflag[np.int_(iind)])
                print('subacc=',subacc[np.int_(iind)])

        if verbose:
            print("number of -1s=",np.size(np.where(pkmassid==-1)))
            print('number of non -1s=',np.size(np.where(pkmassid>-1)))
            print('first_progs_indices_main=',first_progs_indices_main)


        # now we deal with problem cases
        jj, = np.where( (pkmassid == -1) & (fofflag >= 0) )
        for pid in np.unique( rootid[jj] ):
            if verbose:
                print('pid=',pid)
            desc = pid
            cnt = 0
            
            while t.data['snum'][desc] != maxsnap and t.data['desc'][desc] != first_progs_indices_main[ snap_numbers_main==t.data['snum'][t.data['desc'][desc]] ] and t.data['desc'][desc] != -1:

                if cnt > maxsnap: 
                    break

                desc = t.data['desc'][desc]
                cnt += 1
                
                if verbose:
                    print('desc=',desc,t.data['snum'][desc])                
                    print('snum=',t.data['snum'][desc])
                    print('subnum=)',t.data['sbnr'][desc])
                    print('fofnum=',t.data['sgnr'][desc])
                    print('smas=',t.data['smty'][desc][4])

            
            snap_numbers,redshifts,subfind_indices,first_progs_indices,ff_tree_indices,fof_indices,prog_mass,dum = t.return_first_next_mass_progenitors( desc )
            prog_mass = np.array(prog_mass)
            indy = np.argsort(prog_mass)
            if verbose:
                print('first_progs_indices,prog_mass=',first_progs_indices,prog_mass)
                
            try:
                pkmassid[ rootid==pid ] = first_progs_indices[ indy[-1] ]            
            except:
                if verbose:
                    print('Skipping pkmassid here.')
            #pkmassid[ rootid==pid ] = desc


    print('pkmassid_infalltime=',pkmassid_infalltime)
    print('Number of negative pkmassids=',np.size(np.where(pkmassid<0)))
                
    aflag = -9*np.ones(len(idacc))
    indy, = np.where( np.isin(idacc, mainsub_ids)==True )
    aflag[indy] = 0
    indy, = np.where( np.isin(idacc, sub_ids)==True )
    aflag[indy] = 1

    isort = np.argsort(idacc)
    
    idacc_arr = np.array(idacc)[isort]
    subacc_arr = np.array(subacc)[isort]
    snapacc_arr = np.array(snapacc)[isort]
    aflag_arr = np.array(aflag)[isort]
    if t:
        fofflag_arr = np.array(fofflag)[isort]
        rootid_arr = np.array(rootid)[isort]
        pkmassid_arr = np.array(pkmassid)[isort]
        pkmassidtinf_arr = s.cosmology_get_lookback_time_from_a( np.array(pkmassid_infalltime)[isort], is_flat=True)

        print('pkmassidtinf_arr=',pkmassidtinf_arr)
        
    idins_arr = np.array(idins)

    #print('pkmassid_arr=',pkmassid_arr)
    n_acc = len(idacc_arr)
    n_ins = len(idins_arr)


    # Set boundfirst snap array for accreted star particles (aflag = 0, 1)
    
    print('len id_fb, idacc_arr=',len(id_fb),len(idacc_arr))
    indy, = np.where( np.isin(id_fb, idacc_arr)==True )
    bfsnap_arr = boundmain_firstsnap[indy]
    print('len bfsnap_arr =',len(bfsnap_arr))
    #ineg, = np.where( bfsnap_arr < 0 )
    #if len(ineg):
    #    raise ValueError("There is a negative value in the bfsnap_arr.")
    
    print("number of: total accreted,in-situ,aflag[0],aflag[1] (for snap)=",n_acc,n_ins,np.size(aflag[aflag==0]),np.size(aflag[aflag==1]),snap)

    #print('First bound snaps for accreted stars (for snap)=',bfsnap_arr,np.size(bfsnap_arr),snap)

    print('bfsnap_arr=',bfsnap_arr)
    bftime_arr = -1*np.ones(len(bfsnap_arr))
    bftime_arr[bfsnap_arr>=0] = s.cosmology_get_lookback_time_from_a( expansion_fact[np.int_(bfsnap_arr[bfsnap_arr>=0])], is_flat=True )

    print('bftime=',bftime_arr)
    print('expansion_fact=',expansion_fact)
    
    if not writehdf5:
        # write binary
        fout = '%s/%sstarID_accreted_all_newmtree_%03d.dat' % (wpath,run.split("/")[-1],snap)
        fid = open(fout,'wb')
        # write to file
        fid.write(struct.pack('i', n_acc))
        fid.write(struct.pack( '%sQ' % n_acc, *idacc_arr ))
        fid.write(struct.pack( '%sd' % n_acc, *subacc_arr ))
        fid.write(struct.pack( '%sd' % n_acc, *snapacc_arr ))
        fid.write(struct.pack( '%sd' % n_acc, *aflag_arr ))
        if dobfs:
            fid.write(struct.pack( '%sd' % n_acc, *bfsnap_arr ))
        if t:
            fid.write(struct.pack( '%sd' % n_acc, *fofflag_arr ))
            fid.write(struct.pack( '%sd' % n_acc, *rootid_arr ))
            fid.write(struct.pack( '%sd' % n_acc, *pkmassid_arr ))
            fid.write(struct.pack( '%sd' % n_acc, *pkmassidtinf_arr ))
        fid.write(struct.pack('i', n_ins))
        fid.write(struct.pack( '%sQ' % n_ins, *idins_arr ))
        fid.close()

    else:
        #write hdf5
        #fout = '%s/%sstarID_accreted_all_newmtree_%03d.hdf5' % (wpath,run,snap)
        fout = '%s/%sstarID_accreted_all_newmtree_%03d.hdf5' % (wpath,run.split("/")[-1],snap)
        if n_ins > 0 or n_acc > 0:
            with h5py.File(fout, 'w') as f:
                header = f.create_group('Header')
                header.attrs['n_ins'] = n_ins
                header.attrs['n_acc'] = n_acc
                header.attrs['Time'] = Time
                if n_ins > 0:
                    g1 = f.create_group('Insitu')
                    #print('idins_arr=',idins_arr)
                    gg1 = g1.create_dataset('ParticleIDs', (n_ins,), dtype='uint64')
                    gg1[:] = idins_arr
                if n_acc > 0:
                    g2 = f.create_group('Exsitu')
                    gg2 = g2.create_dataset('ParticleIDs', (n_acc,), dtype='uint64')
                    gg2[:] = idacc_arr
                    gg3 = g2.create_dataset('BirthSubhaloindex', (n_acc,), dtype='i4')
                    gg3[:] = subacc_arr
                    #gg4 = g2.create_dataset('BirthTime', (n_acc,), dtype='f8')
                    #gg4[:] = birthtime
                    gg5 = g2.create_dataset('AccretedFlag', (n_acc,), dtype='i4')
                    gg5[:] = aflag_arr
                    if dobfs:
                        gg6 = g2.create_dataset('BoundFirstTime', (n_acc,), dtype='f4')
                        gg6[:] = bftime_arr
                    if t:
                        gg7 = g2.create_dataset('BirthFoFindex', (n_acc,), dtype='i4')
                        gg7[:] = fofflag_arr
                        gg6 = g2.create_dataset('RootIndex', (n_acc,), dtype='i4')
                        gg6[:] = rootid_arr
                        gg7 = g2.create_dataset('PeakMassIndex', (n_acc,), dtype='i4')
                        gg7[:] = pkmassid_arr
                        gg8 = g2.create_dataset('PeakMassInfalltime', (n_acc,), dtype='f4')
                        gg8[:] = pkmassidtinf_arr


            f.close()
    gc.collect()


    print('DONE snap %d'%snap)

In [4]:
# get_accreted_insitu_particles( runs, dirs, outpath, treepaths, mergertree=False, dobfs=True, verbose=False, writehdf5=True )

runs = ['Au26_lvl4_cfea_fiducial/']
dirs = ['./']
outpath = [dirs[0]+runs[0]]
treepaths = [dirs[0]+runs[0]+'trees/']

get_accreted_insitu_particles( runs, dirs, outpath, treepaths, mergertree=True, dobfs=True, verbose=True, writehdf5=True )

<<Doing run Au26_lvl4_cfea_fiducial/ >>
in path  ./
expansion_fact= [0.02093231 0.02194802 0.02301301 0.02401563 0.02518095 0.02640282
 0.02768398 0.02889009 0.03029195 0.03176182 0.03330302 0.034919
 0.03644033 0.03820854 0.04006256 0.04200654 0.04383665 0.04596376
 0.04819409 0.05053264 0.0527342  0.05529306 0.05797608 0.06078929
 0.0634377  0.06651593 0.06974353 0.07312774 0.07667616 0.08001672
 0.08389942 0.08797053 0.09223918 0.09625777 0.10092855 0.10582597
 0.11096103 0.11634526 0.12141409 0.12730555 0.13348287 0.13995995
 0.1460576  0.15314485 0.16057599 0.16836772 0.17653754 0.18422877
 0.19316822 0.20254145 0.21236949 0.22267443 0.2334794  0.24480867
 0.25668768 0.26787083 0.2795412  0.29034102 0.30155808 0.31172794
 0.32224077 0.33310814 0.344342   0.35427208 0.36448853 0.37499959
 0.38398999 0.39506344 0.40453486 0.41423335 0.42416436 0.43433346
 0.44474636 0.45325613 0.46412269 0.47300321 0.48205365 0.49127726
 0.50305535 0.51268081 0.52002058 0.52997065 0.54011111 0.55044

/tmp/ipykernel_2027226/985419296.py:140: RuntimeWarning: invalid value encountered in scalar divide
  dmfrac = subhalodmmass / subhalototmass


Looking in fof 9373 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9373
Looking in fof 9374 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9374
Looking in fof 9375 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9375
Looking in fof 9376 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9376
Looking in fof 9377 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9377
Looking in fof 9378 (out of 41284) with 0 subhaloes (snap 127)
Looking in fof 9379 (out of 41284) with 0 subhaloes (snap 127)
Looking in fof 9380 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9380
Looking in fof 9381 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9381
Looking in fof 9382 (out of 41284) with 0 subhaloes (snap 127)
Looking in fof 9383 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9383
Looking in fof 9384 (out of 41284) with 1 subhaloes (snap 127)
doing sub & fof: 0 9384
Looking in fof 9385 (out of 

: 

: 

: 